# LSST Simulation generation
This notebook is a wrapper for StarFast, which generates simulated images with realistic stellar distributions and spectra, psf and filters, and detailed atmospheric differential chromatic refraction (DCR)

In [1]:
import os
import imp
import numpy as np
import cPickle
import galsim
from lsst.afw.geom import Angle
from lsst.sims.photUtils import matchStar

imp.load_source('calc_refractive_index', '/Users/sullivan/LSST/code/StarFast/calc_refractive_index.py')
imp.load_source('fast_dft', '/Users/sullivan/LSST/code/StarFast/fast_dft.py')
imp.load_source('StarFast', '/Users/sullivan/LSST/code/StarFast/StarFast.py')
from StarFast import StarSim

lsst_lat = Angle(np.radians(-30.244639))
lsst_lon = Angle(np.radians(-70.749417))
lsst_alt = 2663.

Set up the general parameters to use for all of the simulations 

In [2]:
seed = 5  # Seed for the random number generator. Simulations with the same seed are reproduceable
dimension = 1024  # Number of pixels on a side for the simulated image
n_star = 10000  # Number of stars to model in the simulated catalog.
# The catalog covers an area ~4x larger than the area, to allow simulated rotations and dithering observations
pixel_scale = 0.25  # plate scale, in arcseconds/pixel
psf_fwhm = 0.25  # FWHM of the PSF, in arcseconds
hottest_star = 'B'  # Hottest star to include (types are 'OBAFGKMR')
coolest_star = 'M'  # Coolest star to include
wavelength_step = 10  # Wavelength resolution of the spectra and calculation of filter and DCR effects. In nm.
ra_offset = Angle(0)  # Additional offset in RA from the field center, for dithering. In radians as an LSST Angle object
dec_offset = Angle(0)  # Additional offset in Dec from the field center, for dithering. In radians as an LSST Angle object
sky_rotation = 0.0  # Sky rotation angle, in Degrees. I realize this is different than RA and Dec
exposureId = 0  # Unique exposure identification number. Also used as the "OBSID"
instrument_noise = 0.  # Adds noise akin to instrumental noise (post-PSF). Set to 1.0 for default value, can be scaled up or down
photon_noise = 1./15.  # Adds poisson noise akin to photon shot noise. Set to 1.0 for default value, can be scaled up or down
sky_noise = 0  # Adds noise prior to convolving with the PSF.
elevation_min = 30.0  # Minimum observation elevation angle to simulate, in degrees
elevation_max = 90.0  # Open maximum observation angle, in degrees. Only anlges less than elevation_max will be simulated
elevation_step = 5  # Elevation angle step size, in degrees.
output_directory = "/Users/sullivan/LSST/simulations/test%1i/" % seed  # Path to the directory to save output
band_dict = {'u': 0, 'g': 1, 'r': 2, 'i': 3, 'z': 4, 'y': 5}  # LSST filter numbers used by the butler

In [3]:
ra = lsst_lon + ra_offset
dec = lsst_lat + dec_offset

Load simulated Kurucz stellar SEDs. Cache the list for later use.

In [4]:
pickle_file = 'sed_list.pickle'
if os.path.exists(pickle_file):
    with open(pickle_file, 'rb') as dumpfile:
        sed_list = cPickle.load(dumpfile)
else:
    matchStarObj = matchStar()
    sed_list = matchStarObj.loadKuruczSEDs()
    with open(pickle_file, 'wb') as dumpfile:
        cPickle.dump(sed_list, dumpfile, cPickle.HIGHEST_PROTOCOL)

Set up the PSF.

In [5]:
gsp = galsim.GSParams(folding_threshold=1.0 / (dimension), maximum_fft_size=12288)
psf = galsim.Kolmogorov(fwhm=psf_fwhm / pixel_scale, flux=1, gsparams=gsp)

# u-band simulation

In [6]:
band_name = 'u'

In [7]:
sim = StarSim(psf=psf, pixel_scale=pixel_scale, x_size=dimension, y_size=dimension,
              band_name=band_name, wavelength_step=wavelength_step,
              sed_list=sed_list, ra=ra, dec=dec, sky_rotation=sky_rotation)

Simulate a catalog of stars, with fluxes and SEDs

In [8]:
sim.load_catalog(n_star=n_star, hottest_star=hottest_star, coolest_star=coolest_star, seed=seed)

Number and flux contribution of stars of each type:
 [M 7642| 1.85%] [K 1213| 2.21%] [G 751| 4.25%] [F 329| 5.55%] [A 60| 11.72%] [B 5| 74.43%] [O 0| 0.00%]


Write the catalog to disk. Note that this catalog includes fluxes for all 6 LSST bands if filter_list is None

In [9]:
sim.make_reference_catalog(output_directory=output_directory + "and_files/", filter_list=None, magnitude_limit=16.0)

Writing 1917 stars brighter than 16.0 mag to reference catalog in 6 bands
Min/max magnitude:  4.6198589147 20.8877358187


Generate the raw simulation

In [9]:
sim.simulate()

Simulating 2464 stars within observable region
Time to model 2458 stars: [6.601s | 0.00269s per star]
Time to model 6 bright stars: [5.130s | 0.85507s per star]


In [10]:
from lsst.sims.photUtils import Bandpass, matchStar, PhotometricParameters
photo = PhotometricParameters(exptime=30., nexp=1, platescale=0.25, bandpass='g')

In [11]:
expId = exposureId + 100*band_dict[band_name]
for elevation in np.arange(elevation_min, elevation_max, elevation_step):
    for azimuth in [0.0, 180.0]:
        exposure = sim.convolve(elevation=elevation, azimuth=azimuth,
                                instrument_noise=instrument_noise, sky_noise=sky_noise,
                                photon_noise=photon_noise, exposureId=expId, obsid=expId)
        filename = "lsst_e_%3.3i_f%i_R22_S11_E000.fits" % (expId, band_dict[band_name])
        expId += 1
        exposure.writeFits(output_directory + "images/" + filename)

FFT timing for 8 DCR planes: [2.235s | 0.279s per plane]
FFT timing for 8 DCR planes: [7.801s | 0.975s per plane]
FFT timing for 8 DCR planes: [1.987s | 0.248s per plane]
FFT timing for 8 DCR planes: [8.071s | 1.009s per plane]
FFT timing for 8 DCR planes: [1.994s | 0.249s per plane]
FFT timing for 8 DCR planes: [8.146s | 1.018s per plane]
FFT timing for 8 DCR planes: [1.935s | 0.242s per plane]
FFT timing for 8 DCR planes: [7.726s | 0.966s per plane]
FFT timing for 8 DCR planes: [1.896s | 0.237s per plane]
FFT timing for 8 DCR planes: [7.846s | 0.981s per plane]
FFT timing for 8 DCR planes: [1.883s | 0.235s per plane]
FFT timing for 8 DCR planes: [7.856s | 0.982s per plane]
FFT timing for 8 DCR planes: [1.970s | 0.246s per plane]
FFT timing for 8 DCR planes: [7.720s | 0.965s per plane]
FFT timing for 8 DCR planes: [1.891s | 0.236s per plane]
FFT timing for 8 DCR planes: [7.724s | 0.965s per plane]
FFT timing for 8 DCR planes: [1.964s | 0.246s per plane]
FFT timing for 8 DCR planes: [8

# g-band simulation

In [12]:
band_name = 'g'

In [13]:
sim = StarSim(psf=psf, pixel_scale=pixel_scale, x_size=dimension, y_size=dimension,
              band_name=band_name, wavelength_step=wavelength_step,
              sed_list=sed_list, ra=ra, dec=dec, sky_rotation=sky_rotation)

Simulate a catalog of stars, with fluxes and SEDs

In [14]:
sim.load_catalog(n_star=n_star, hottest_star=hottest_star, coolest_star=coolest_star, seed=seed)

Number and flux contribution of stars of each type:
 [M 7642| 1.85%] [K 1213| 2.21%] [G 751| 4.25%] [F 329| 5.55%] [A 60| 11.72%] [B 5| 74.43%] [O 0| 0.00%]


Generate the raw simulation

In [15]:
sim.simulate()

Simulating 2464 stars within observable region
Time to model 2459 stars: [9.178s | 0.00373s per star]
Time to model 5 bright stars: [7.256s | 1.45120s per star]


In [16]:
expId = exposureId + 100*band_dict[band_name]
for elevation in np.arange(elevation_min, elevation_max, elevation_step):
    for azimuth in [0.0, 180.0]:
        exposure = sim.convolve(elevation=elevation, azimuth=azimuth,
                                instrument_noise=instrument_noise, sky_noise=sky_noise,
                                photon_noise=photon_noise, exposureId=expId, obsid=expId)
        filename = "lsst_e_%3.3i_f%i_R22_S11_E000.fits" % (expId, band_dict[band_name])
        expId += 1
        exposure.writeFits(output_directory + "images/" + filename)

FFT timing for 15 DCR planes: [3.524s | 0.235s per plane]
FFT timing for 15 DCR planes: [13.738s | 0.916s per plane]
FFT timing for 15 DCR planes: [3.303s | 0.220s per plane]
FFT timing for 15 DCR planes: [13.493s | 0.900s per plane]
FFT timing for 15 DCR planes: [3.308s | 0.221s per plane]
FFT timing for 15 DCR planes: [13.447s | 0.896s per plane]
FFT timing for 15 DCR planes: [3.910s | 0.261s per plane]
FFT timing for 15 DCR planes: [14.701s | 0.980s per plane]
FFT timing for 15 DCR planes: [3.700s | 0.247s per plane]
FFT timing for 15 DCR planes: [14.812s | 0.987s per plane]
FFT timing for 15 DCR planes: [3.606s | 0.240s per plane]
FFT timing for 15 DCR planes: [14.301s | 0.953s per plane]
FFT timing for 15 DCR planes: [3.316s | 0.221s per plane]
FFT timing for 15 DCR planes: [14.365s | 0.958s per plane]
FFT timing for 15 DCR planes: [3.364s | 0.224s per plane]
FFT timing for 15 DCR planes: [13.757s | 0.917s per plane]
FFT timing for 15 DCR planes: [3.351s | 0.223s per plane]
FFT ti